In [25]:
%reload_ext autoreload
%autoreload 2

In [26]:
from fastai.basics import *

In [27]:
PATH = Path('data/rossmann')

In [28]:
PATH

PosixPath('data/rossmann')

In [29]:
PATH.ls()

[PosixPath('data/rossmann/googletrend.csv'),
 PosixPath('data/rossmann/weather.csv'),
 PosixPath('data/rossmann/rossmann.tgz'),
 PosixPath('data/rossmann/test.csv'),
 PosixPath('data/rossmann/train.csv'),
 PosixPath('data/rossmann/state_names.csv'),
 PosixPath('data/rossmann/sample_submission.csv'),
 PosixPath('data/rossmann/store_states.csv'),
 PosixPath('data/rossmann/store.csv')]

In [30]:

table_names = ['train', 'store', 'store_states', 'state_names', 'googletrend', 'weather', 'test']
tables = [pd.read_csv(PATH/f'{fname}.csv', low_memory=False) for fname in table_names]
train, store, store_states, state_names, googletrend, weather, test = tables
len(train), len(test)

(1017209, 41088)